### Experiment Structured output with RAG pipeline

In [1]:
import openai
import instructor
from qdrant_client import QdrantClient
from pydantic import BaseModel, Field

### Import RAG Pipeline

In [ ]:
from qdrant_client import QdrantClient
import openai


def create_embeddings(text, model="text-embedding-3-small"):
   
    response = openai.embeddings.create(
        model=model,
        input=text
    )
        
    return response.data[0].embedding

def retrieve_embedding_data(qd_client: QdrantClient, query, collection_name, k=5):
    response = qd_client.query_points(
        collection_name=collection_name,
        query=create_embeddings(query),
        limit=k
    )

    retrieved_context_ids = []
    retrieved_context = []
    retrieved_scores = []
    retrieved_context_ratings = []
    
    for point in response.points:
        retrieved_context_ids.append(point.payload["parent_asin"])
        retrieved_context.append(point.payload["description"])
        retrieved_scores.append(point.score)
        retrieved_context_ratings.append(point.payload["average_rating"])

    # return dictionary of retrieved data
    return {
        "context_ids": retrieved_context_ids,
        "context": retrieved_context,
        "scores": retrieved_scores,
        "context_ratings": retrieved_context_ratings
    }


def format_context(retrived_context):
    formatted_context = ""
    for id, chunk, rating in zip(retrived_context["context_ids"], retrived_context["context"], retrived_context["context_ratings"]):
        formatted_context += f"Product ID: {id}, rating: {rating}, description: {chunk.strip()}\n"
    return formatted_context

def build_prompt(preprocessed_context, question):
    prompt = f"""
You are a specialized Product Expert Assistant. Your goal is to answer customer questions accurately using ONLY the provided product information.

### Instructions:
1. **Source of Truth:** Answer strictly based on the provided "Available Products" section below. Do not use outside knowledge or make assumptions.
2. **Handling Missing Info:** If the answer cannot be found in the provided products, politely state that you do not have that information. Do not make up features.
3. **Tone:** Be helpful, professional, and concise.
4. **Terminology:** Never refer to the text below as "context" or "data." Refer to it naturally as "our current inventory" or "available products."

### Available Products:
<inventory_data>
{preprocessed_context}
</inventory_data>

### Customer Question:
{question}

### Answer:
"""
    return prompt


def generate_llm_response(prompt, model="gpt-5-nano"):
    
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
        ]
    )
    
    return response.choices[0].message.content

def integrated_rag_pipeline(question, model="gpt-5-nano"):
    
    qdrant_client = QdrantClient(   
        url="http://localhost:6333",
    )
    # Step 1: Retrieve relevant context
    retrieved_context = retrieve_embedding_data(
        qdrant_client,
        question,
        collection_name="amazon_items-collection-00",
        k=5
    )
    # Step 2: Format context
    formatted_context = format_context(retrieved_context)   
    # Step 3: Build prompt
    prompt = build_prompt(formatted_context, question)
    # Step 4: Generate response
    response = generate_llm_response(prompt, model)
    
    final_response = {
        "question": question,
        "answer": response,
        "retrieved_context_ids": retrieved_context["context_ids"],
        "retrieved_context": retrieved_context["context"],
        "similarity_scores": retrieved_context["scores"],
    }
        
    return final_response

In [ ]:
response = integrated_rag_pipeline("What is the best mobile phone product in the market?")

In [6]:
response

{'question': 'What is the best mobile phone product in the market?',
 'answer': 'I don’t have information on the best mobile phone product in the market. Our current inventory does not include any mobile phones.\n\nIf you’d like, I can help you with options related to mobile usage from our available products:\n- B0CG8J2X6X — Aewittor Android Auto Wireless Car Adapter\n- B0BWN4MYDK — Smart Watch (Bluetooth calling/health tracking)\n- B0BCQ8RJG7 — Fullant Kids Tablet 7 inch\n- B0B5517LRN — Android 12.0 Tablet 10 inch\n- B0BM71PJQ9 — Sanag Bluetooth Speakers\n\nTell me what you’re looking for (driving convenience, fitness tracking, kids’ device, general tablet use, audio), and I’ll help you compare these options.',
 'retrieved_context_ids': ['B0CG8J2X6X',
  'B0BWN4MYDK',
  'B0BCQ8RJG7',
  'B0B5517LRN',
  'B0BM71PJQ9'],
 'retrieved_context': ['Aewittor Newest Wireless Android Auto Car Adapter- Fast, Plug & Play, Turn Wired to Wireless - Suitable Dongle for OEM Android Auto Cars & 2016+ [\'

In [63]:
from typing import List

class RAGContext(BaseModel):
    id: str = Field(description="The id of the product used to answer the question")
    description: str = Field(description="The short description of the product used to answer the question")

class RAGResponse(BaseModel):
    answer: str = Field(description="The answer to the question")
    references: List[RAGContext] = Field(description="List of RAG Context items used to answer the question")
    

    

In [64]:
from qdrant_client import QdrantClient
import openai

instructor_prompt = instructor.from_openai(openai.OpenAI())

def create_embeddings(text, model="text-embedding-3-small"):
   
    response = openai.embeddings.create(
        model=model,
        input=text
    )
        
    return response.data[0].embedding

def retrieve_embedding_data(qd_client: QdrantClient, query, collection_name, k=5):
    response = qd_client.query_points(
        collection_name=collection_name,
        query=create_embeddings(query),
        limit=k
    )

    retrieved_context_ids = []
    retrieved_context = []
    retrieved_scores = []
    retrieved_context_ratings = []
    
    for point in response.points:
        retrieved_context_ids.append(point.payload["parent_asin"])
        retrieved_context.append(point.payload["description"])
        retrieved_scores.append(point.score)
        retrieved_context_ratings.append(point.payload["average_rating"])

    # return dictionary of retrieved data
    return {
        "context_ids": retrieved_context_ids,
        "context": retrieved_context,
        "scores": retrieved_scores,
        "context_ratings": retrieved_context_ratings
    }


def format_context(retrived_context):
    formatted_context = ""
    for id, chunk, rating in zip(retrived_context["context_ids"], retrived_context["context"], retrived_context["context_ratings"]):
        formatted_context += f"Product ID: {id}, rating: {rating}, description: {chunk.strip()}\n"
    return formatted_context

def build_prompt(preprocessed_context, question):
    prompt = f"""
You are a specialized Product Expert Assistant. Your goal is to answer customer questions accurately using ONLY the provided product information.

### Instructions:
1. **Source of Truth:** Answer strictly based on the provided "Available Products" section below. Do not use outside knowledge or make assumptions.
2. **Handling Missing Info:** If the answer cannot be found in the provided products, politely state that you do not have that information. Do not make up features.
3. **Tone:** Be helpful, professional, and concise.
4. **Terminology:** Never refer to the text below as "context" or "data." Refer to it naturally as "our current inventory" or "available products."
5. **output format** - An output of the following format is expected:
    
    1. **Answer:** The answer to the question.
    2. **Context:** The list of the IDs of the chunks that were used to answer the question. Only return the ones that are used in the answer.
    3. **Description:** Short description (1-2 sentences) of the item based on the description provided in the context.

### Available Products:
<inventory_data>
{preprocessed_context}
</inventory_data>

### Customer Question:
{question}

### Answer:
"""
    return prompt


def generate_llm_response(prompt, model="gpt-4.1-mini"):
    
    response, raw_response = instructor_prompt.chat.completions.create_with_completion(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
        ],
        response_model=RAGResponse
    )
    
    return response

def integrated_rag_pipeline(question, model="gpt-5-nano"):
    
    qdrant_client = QdrantClient(   
        url="http://localhost:6333",
    )
    # Step 1: Retrieve relevant context
    retrieved_context = retrieve_embedding_data(
        qdrant_client,
        question,
        collection_name="amazon_items-collection-00",
        k=5
    )
    # Step 2: Format context
    formatted_context = format_context(retrieved_context)   
    # Step 3: Build prompt
    prompt = build_prompt(formatted_context, question)
    # Step 4: Generate response
    response = generate_llm_response(prompt, model)
    
    final_response = {
        "question": question,
        "answer": response,
        "retrieved_context_ids": retrieved_context["context_ids"],
        "retrieved_context": retrieved_context["context"],
        "similarity_scores": retrieved_context["scores"],
    }
        
    return final_response

In [65]:
response = integrated_rag_pipeline("give me head phones?")

In [66]:
response

{'question': 'give me head phones?',
 'answer': RAGResponse(answer='Here are headphone options from our available products.', references=[RAGContext(id='B0BNHVLF7G', description='Siniffo Upgraded Bone Conduction Headphones with Bluetooth 5.3, open-ear design, sweat resistant, suitable for workouts and commuting.'), RAGContext(id='B0BXH6XVY9', description='LORELEI B-C6 Wireless Over Ear Headphones with 50 hours playtime and Bluetooth 5.3, memory foam ear cushions.'), RAGContext(id='B0B2P18Z31', description='DOQAUS Wired Headphones for Kids with volume limiter and Shareport, 3.5mm jack compatibility.'), RAGContext(id='B0C9V31PL1', description='Wireless Earbuds Sports Bluetooth Headphones with 4 mics noise cancellation, up to 60 hours play time with charging case.'), RAGContext(id='B0C6PGF8BV', description='Apple Earbuds with Lightning Connector, wired in-ear headphones with built-in remote for iOS devices.')]),
 'retrieved_context_ids': ['B0BNHVLF7G',
  'B0BXH6XVY9',
  'B0B2P18Z31',
  'B

In [67]:
response["answer"].references

[RAGContext(id='B0BNHVLF7G', description='Siniffo Upgraded Bone Conduction Headphones with Bluetooth 5.3, open-ear design, sweat resistant, suitable for workouts and commuting.'),
 RAGContext(id='B0BXH6XVY9', description='LORELEI B-C6 Wireless Over Ear Headphones with 50 hours playtime and Bluetooth 5.3, memory foam ear cushions.'),
 RAGContext(id='B0B2P18Z31', description='DOQAUS Wired Headphones for Kids with volume limiter and Shareport, 3.5mm jack compatibility.'),
 RAGContext(id='B0C9V31PL1', description='Wireless Earbuds Sports Bluetooth Headphones with 4 mics noise cancellation, up to 60 hours play time with charging case.'),
 RAGContext(id='B0C6PGF8BV', description='Apple Earbuds with Lightning Connector, wired in-ear headphones with built-in remote for iOS devices.')]